In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [6]:
scales = [8]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.median(x)

error = 0.0001
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-24884,24986],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":12053,"total_const_size":1816,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 12.05522108078003 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.55078125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18341455175509539295, 12796101019039945164, 1607286914885633240, 1929881192315725821]]], 'proof': '1855e44b44516d4d9c5bfadbd2fe4399875ffccd667f28f86246f8c00d2cc18b01ed269be180d1899d595710036c63da0d01da1df683cdd7c066c06b44f0c37618b8eca1dd85203d254fc5a1a183353acf4719c85f9241fbfb7def0d939339331d8cd5f67e5610d5dc22d54db26e2646adf0def1519a0f4bf95ee57fd044464d2d2a8cb026954177fd08d139bc6ef98e5bc57ad8cc00149198134eeb0b58b87418353df4bbf72603ac3e20508049d25bc918a8cd2ea689e2158decd7560f585427e88ebde550f7ce1cacff4ee8d92d6777d34683a1df24f8b756f5531b925d46167676f7109bb990cb0dd4705cb1ad5a276fc6604fe513866df4cb5799dc044014f1c9bb8e4e30c6fe736625e5bdf3821274aab29a1ef5fdeac3bba339ea417311cdaa313fe068b20b010a6a81e7447ce79ace746914ce90454120890d34ed230ac49a2b3d74f4d6add7bf00bda28e938beb17baa300a2e845696f9d6e12b997

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 49.55078125
